In [ ]:
using Pkg
if isfile("../Project.toml") && isfile("../Manifest.toml")
    Pkg.activate("..");
    ENV["PYTHON"] = "python3";
end

using Plots, Random, Distributions, LinearAlgebra
import Plots: Plot
import JuliaProbo: AbstractObject, AbstractWorld, AbstractAgent, AbstractEstimator, Landmark, Map, KalmanFilter, EstimatorAgent, RealRobot, RealCamera, World, Goal, Puddle, PuddleWorld, puddle_depth, observations, push!, draw, decision, state_transition, motion_update, observation_update
gr();

In [ ]:
mutable struct PuddleIgnoreAgent <: AbstractAgent
    v_::Float64
    ω_::Float64
    dt::Float64
    prev_v_::Float64
    prev_ω_::Float64
    estimator_::AbstractEstimator
    puddle_coeff::Float64
    puddle_depth_::Float64
    total_reward_::Float64
    function PuddleIgnoreAgent(
        v::Float64,
        ω::Float64,
        dt::Float64,
        estimator::AbstractEstimator;
        puddle_coeff=100
    )
        new(v, ω, dt, 0.0, 0.0, estimator, puddle_coeff, 0.0, 0.0)
    end
end

function reward_per_sec(agent::PuddleIgnoreAgent)
    return -1.0 - agent.puddle_depth_ * agent.puddle_coeff
end

function decision(agent::PuddleIgnoreAgent, observation::Vector{Vector{Float64}})
    motion_update(agent.estimator_,
        agent.prev_v_,
        agent.prev_ω_,
        agent.dt)
    obseravtion = Vector{Vector{Float64}}(undef, 0)
    agent.prev_v_, agent.prev_ω_ = agent.v_, agent.ω_
    observation_update(agent.estimator_, observation)
    agent.total_reward_ += agent.dt * reward_per_sec(agent)
    return agent.v_, agent.ω_
end

function draw(agent::PuddleIgnoreAgent, p::Plot{T}) where {T}
    x, y = agent.estimator_.pose_[1], agent.estimator_.pose_[2]
    annota1 = "reward/sec: $(round(reward_per_sec(agent), sigdigits=3))"
    annota2 = "total reward: $(round(agent.total_reward_, sigdigits=3))"
    p = annotate!(x+1.0, y-0.5, text(annota1, 10))
    p = annotate!(x+1.0, y-1.0, text(annota2, 10))
end

function update_reward(world::PuddleWorld)
    for iter in 1:lastindex(world.robots_)
        robot = world.robots_[iter]
        # use the ground truth position
        robot.agent_.puddle_depth_ = puddle_depth(world, robot.pose_)
    end
end

function ch10_puddle_world3()
    dt = 0.1
    # environment
    xlim = [-5.0, 5.0]
    ylim = [-5.0, 5.0]
    # id of landmark must start from 0 with 1 step
    landmarks =
        [Landmark([-4.0, 2.0], 0), Landmark([2.0, -3.0], 1), Landmark([3.0, 3.0], 2)]
    envmap = Map()
    push!(envmap, landmarks)
    world = PuddleWorld(xlim, ylim)
    push!(world, envmap)
    # robot side
    initial_pose = [0.0, 0.0, 0.0]
    estimator = KalmanFilter(envmap, initial_pose)
    agent = PuddleIgnoreAgent(0.2, 10.0 * pi / 180, dt, estimator)
    robot = RealRobot(initial_pose, agent, RealCamera(landmarks, distance_bias_rate_stddev = 0.0, direction_bias_stddev = 0.0); color = "red")
    push!(world, robot)
    # goal
    goal = Goal(-3.0, -3.0)
    push!(world, goal)
    # puddles
    push!(world, Puddle([-2.0, 0.0], [0.0, 2.0], 0.1))
    push!(world, Puddle([-0.5, -2.0], [2.5, 1.0], 0.1))
    
    anim = @animate for i = 1:400
        t = dt * i
        annota = "t = $(round(t, sigdigits=3))[s]"
        # t
        update_reward(world)
        z = observations(robot.sensor_, robot.pose_; noise = false, bias = false)
        p = draw(world, annota)
        
        # t+1
        v, ω = decision(agent, z)
        state_transition(robot, v, ω, dt; move_noise = true, vel_bias_noise = true)
    end
    gif(anim, "images/ch10_puddle_world3.gif", fps = 20)
end
ch10_puddle_world3()

<img src="images/ch10_puddle_world3.gif">